# **Constitutency grammer & Dependency parsing**

In [1]:
import nltk
# nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown.zip.
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown_tei.zip.
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_cat.zip.
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_esp.zip.
[nltk_data]    | Downloading package chat80 to /root/nltk_data...
[nltk_data]    |   Unzipp

True

**1. CFG(Context-Free-Grammer) :: Phrase-Structure Grammers**

In [7]:
#ex1 

#set of the rules(productions) :expression of the symbols 
groucho_grammar = nltk.CFG.fromstring("""
S -> NP VP
PP -> P NP
NP -> Det N | Det N PP | 'I'
VP -> V NP | VP PP
Det -> 'an' | 'my'
N -> 'elephant' | 'pajamas'
V -> 'shot' | 'have'
P -> 'in'
""")

sent = 'I shot an elephant in my pajamas'.split() #need to split to elements of list 
parser = nltk.ChartParser(groucho_grammar)
for tree in parser.parse(sent):
  print(tree)
#outputs two structures :: ambiguous 

(S
  (NP I)
  (VP
    (VP (V shot) (NP (Det an) (N elephant)))
    (PP (P in) (NP (Det my) (N pajamas)))))
(S
  (NP I)
  (VP
    (V shot)
    (NP (Det an) (N elephant) (PP (P in) (NP (Det my) (N pajamas))))))


format is :: 'syntatic category' -> 'element' 

Can create a phrase structure tree making the nodes as the constitutnets of the sentence 

Does not think about the context, thinks of the structure around each words :: with terminal, non-terminals 

S is the start of the senetence 


In [9]:
#ex2 

#grammar definition 
grammar1 = nltk.CFG.fromstring("""
  S -> NP VP
  VP -> V NP | V NP PP
  PP -> P NP
  V -> "saw" | "ate" | "walked"
  NP -> "John" | "Mary" | "Bob" | Det N | Det N PP
  Det -> "a" | "an" | "the" | "my"
  N -> "man" | "dog" | "cat" | "telescope" | "park"
  P -> "in" | "on" | "by" | "with"
  """)

#parsing 
sent = "Mary saw Bob".split()
rd_parser = nltk.RecursiveDescentParser(grammar1)
for tree in rd_parser.parse(sent):
  print(tree)

(S (NP Mary) (VP (V saw) (NP Bob)))


***Parsing in CFG***

1) top-down-method:: recursive descent parsing 

2) bottom-up-method:: shift-reduct parsing 

In [12]:
#top-down method
#start from S and go down till last element (find subgoals recursivly)

rd_parser = nltk.RecursiveDescentParser(grammar1) #recursivly done 

sent = 'Mary saw a dog'.split()

for tree in rd_parser.parse(sent):
     print(tree)

# nltk.app.rdparser()

(S (NP Mary) (VP (V saw) (NP (Det a) (N dog))))


In [14]:
#bottom-up method 
#starts with elements not S :: push the next input word onto the stack repeativly 
#not always find valid parse 
sr_parser = nltk.ShiftReduceParser(grammar1)
sent = 'Mary saw a dog'.split()
for tree in sr_parser.parse(sent):
  print(tree)

(S (NP Mary) (VP (V saw) (NP (Det a) (N dog))))


**2. Dependency grammer:: dependency parsing**

looking through directed relations between lexical items in the sentence 

focus on how words relate to other words (not syntex) 


In [16]:
#ex1 
#the elements of grammar have a relationship of head-dependent 
groucho_dep_grammar = nltk.DependencyGrammar.fromstring("""
'shot' -> 'I' | 'elephant' | 'in'
'elephant' -> 'an' | 'in'
'in' -> 'pajamas'
'pajamas' -> 'my'
 """)

pdp = nltk.ProjectiveDependencyParser(groucho_dep_grammar)

sent = 'I shot an elephant in my pajamas'.split()
trees = pdp.parse(sent)
for tree in trees:
  print(tree)

(shot I (elephant an (in (pajamas my))))
(shot I (elephant an) (in (pajamas my)))


**3. Problems of both parsing**

have ambiguity in splitting the elements of the sentence 

can be various syntax :: example fish(N), fish(V) 

use weighted grammer that gives the probability of sent 


In [17]:
grammar = nltk.PCFG.fromstring("""
    S    -> NP VP              [1.0]
    VP   -> TV NP              [0.4]
    VP   -> IV                 [0.3]
    VP   -> DatV NP NP         [0.3] #gives the probability of each available option
    TV   -> 'saw'              [1.0]
    IV   -> 'ate'              [1.0]
    DatV -> 'gave'             [1.0]
    NP   -> 'telescopes'       [0.8]
    NP   -> 'Jack'             [0.2]
    """)

viterbi_parser = nltk.ViterbiParser(grammar)
for tree in viterbi_parser.parse(['Jack', 'saw', 'telescopes']):
  print(tree)
  

(S (NP Jack) (VP (TV saw) (NP telescopes))) (p=0.064)
